In [1]:
import pandas as pd
df1 = pd.read_csv('Data/rec.csv', sep=';')

In [34]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
class GoogleMaps:
    def __init__(self):
        # Remplacez le chemin vers le driver de votre navigateur
        driver_path = 'chromedriver'
        service = Service(executable_path=driver_path)
        self.driver = webdriver.Chrome(service=service)

    def search_location(self, location):
        # Ouvrir Google Maps
        self.driver.get('https://www.google.com/maps/')
        skip = self.driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button/span').click()
        search_box = self.driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
        search_box.send_keys(location)
        search_box.send_keys(Keys.ENTER)
        time.sleep(2)
        search_box.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]').click()
        time.sleep(2)
        for i in range(1, 101):
            xpath = f'/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[{i}]/div/div/div[4]/div[2]/div/span[2]/button'
            try:
                search_box.find_element(By.XPATH, xpath).click()
            except:
                pass



class TextInformation:
    def __init__(self, url):
        self.url = url
        self.soup = BeautifulSoup(
            url.page_source,
            'html.parser',@@@
            from_encoding='utf-8')

    def get_name(self):
        elements = self.soup.select("div.m6QErb div.MyEned")
        names = [elem.text for elem in elements]
        return names

maps = GoogleMaps()
for row in df1.iterrows():
    location = row[1]['GM_search']
    print(location)
    maps.search_location(location)
    text_info = TextInformation(maps.driver).get_name()
    print(text_info)

144 Rue de Bavay, Réseau des rectorats et DSDEN, Rectorat, RECTORAT DE L'ACADEMIE DE LILLE, LILLE
['J’ai travaillé en collège en juin et juillet 2022, depuis le 3 janvier je réclame mon certificat de travail (qui aurait du être envoyé à la fin de mon contrat, c’est une obligation) et depuis le 3 janvier on se moque clairement de moi et je n’ai toujours pas ce document alors que j’en ai absolument besoin. Je ne sais plus quoi faire, il est temps d’apprendre à faire votre travail! ', 'J’ai du à appeler 4 fois des l’ouverture du standard mais une fois décroché la dame a l’accueil a était super gentil et la dirigé vers son collègue qui m’a correctement renseigné tout en étant poli et agréable. Encore merci ☺️ ', "Toutes mes condoléances à la famille de ce jeune homme. Aucun mot, de la colère et des larmes suite à la gestion d'urgence vitale inacceptable au Lycée Gaston Berger de Lille. Veuillez prendre toutes les mesures nécessaires dans le cadre d'une enquête administrative et mesures adé

In [1]:
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[10]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[13]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[10]/div/div/div[4]/div[2]/div/span[2]/button``
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[16]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[31]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[28]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[31]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[31]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[10]/div/div/div[4]/div[2]/div/span[2]/button
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[10]/div[31]/div/div/div[4]/div[2]/div/span[2]/button

Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
Hello 8
Hello 9
